In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.utils import shuffle

subset100 = pd.read_csv("../raw_data/track_meta_100subset_new.csv")
subset100 = shuffle(subset100)
# subset100 = subset100.reset_index()

### Train-val-test split

In [4]:
subset100.head()

,Playlistid,Trackid,Artist_Name,Track_Name,Album_Name,Track_Duration,Artist_uri,Track_uri,Album_uri,acousticness,...,loudness,mode,speechiness,tempo,time_signature,valence,Playlist,Album,Track,Artist
1538,155598,27,Enrique Iglesias,Bailando - Spanish Version,SEX AND LOVE,243413,spotify:artist:7qG3b048QCHVRO5Pv1T5lw,spotify:track:32lm3769IRfcnrQV11LO4E,spotify:album:2kZkiVn1m00XcgaWlLb2LD,0.04260,...,-3.503,1,0.1080,91.017,4,0.961,Lovey.,44,45,42
1428,153929,13,Fetty Wap,679 (feat. Remy Boyz),679 (feat. Remy Boyz),196693,spotify:artist:6PXS4YHDkKvl1wkIl4V8DL,spotify:track:5NQbUaeTEOGdD6hHcre0dZ,spotify:album:0TyDTzG2zt2tg0wONT9wSU,0.00256,...,-5.738,1,0.3180,190.050,4,0.603,All songs,66,83,41
2807,241546,66,Daughtry,It's Not Over,Daughtry (Deluxe Edition),215026,spotify:artist:5P5FTygHyx2G57oszR3Wot,spotify:track:234RqTZmnDTnxWlVciXHLD,spotify:album:6ii8Ja21xLGkFH8CQ6bKjj,0.05030,...,-3.245,0,0.0618,145.927,4,0.294,good old songs,99,122,82
1103,97870,3,Alexandra Stan,Mr. Saxobeat - Radio Edit,Saxobeats,195280,spotify:artist:0BmLNz4nSLfoWYW1cYsElL,spotify:track:4zIaXqKCXZs02eB6jEe5Mf,spotify:album:11UM41JGr8BavupgcGCICa,0.02390,...,-4.162,0,0.0527,127.047,4,0.745,Gym,6,6,6
1130,107065,10,Digital Farm Animals,Millionaire - Alan Walker Remix,Millionaire,189346,spotify:artist:5fyDppLDl1juIu1BcUT5zh,spotify:track:5HzbxweVAtisCrpoOF11qt,spotify:album:05XuMZb14sDGgreCf3Wu6F,0.01860,...,-4.577,1,0.0545,96.044,4,0.424,feels,11,11,11


In [5]:
# Train-val-test split (20%)
train, test = train_test_split(subset100, test_size=0.2, random_state=42, stratify = subset100['Playlistid'])
train, val = train_test_split(train, test_size=0.2, random_state=42, stratify = train['Playlistid'])

In [6]:
# Drop features here
features_drop = ["Playlistid","Playlist","Album", "Track", "Artist", "Trackid", "Artist_Name", "Track_Name", "Album_Name", "Artist_uri", "Track_uri", "Album_uri", "artist_genres", "explicit"]
train_cleaned, val_cleaned, test_cleaned = train.drop(features_drop, axis =1), val.drop(features_drop, axis=1), test.drop(features_drop, axis=1)

In [7]:
train = train.reset_index(drop=True)

In [8]:
train_cleaned = train_cleaned.reset_index(drop=True)

In [9]:
train_cleaned.head()

,Track_Duration,acousticness,artist_popularity,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,189768,0.90900,100,0.468,0.693,0.000000,6,0.116,-7.059,0,0.5530,84.762,4,0.308
1,226632,0.03060,68,0.690,0.502,0.000000,0,0.145,-7.077,0,0.0638,86.185,4,0.329
2,176561,0.00346,81,0.723,0.809,0.001230,7,0.565,-3.081,0,0.0625,98.007,4,0.274
3,241688,0.63400,73,0.566,0.664,0.000000,4,0.116,-5.303,0,0.0464,128.945,4,0.437
4,448533,0.13100,81,0.690,0.718,0.000729,4,0.615,-4.743,0,0.1350,76.007,4,0.401


### Content-based Collaborative Filtering

In [10]:
# train_content
from sklearn.metrics.pairwise import cosine_similarity 

### Create a cosine-similarity matrix

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_cleaned)
train_scaled = scaler.transform(train_cleaned)
test_scaled = scaler.transform(test_cleaned)

In [12]:
train_scaled_cos_matrix = cosine_similarity(train_scaled)
# np.dot(np.array(trial_scaled[0,]), np.array(trial_scaled[5,]))/ (np.linalg.norm(trial_scaled[0,]) * np.linalg.norm(trial_scaled[5,]))


In [21]:
train_scaled_cos_matrix.shape

(1970, 1970)

In [13]:
train_scaled_cos_matrix[:5]

array([[1.        , 0.77886315, 0.79322869, ..., 0.75797626, 0.89769807,
        0.86462959],
       [0.77886315, 1.        , 0.91136461, ..., 0.74585765, 0.8543402 ,
        0.93407908],
       [0.79322869, 0.91136461, 1.        , ..., 0.7696172 , 0.86108348,
        0.95182369],
       [0.89793126, 0.90315705, 0.88722029, ..., 0.85739621, 0.96830316,
        0.97063001],
       [0.81008439, 0.93095577, 0.95230958, ..., 0.75950365, 0.85586662,
        0.92814526]])

In [20]:
display(subset100.head())
subset100.shape

,Playlistid,Trackid,Artist_Name,Track_Name,Album_Name,Track_Duration,Artist_uri,Track_uri,Album_uri,acousticness,...,loudness,mode,speechiness,tempo,time_signature,valence,Playlist,Album,Track,Artist
1538,155598,27,Enrique Iglesias,Bailando - Spanish Version,SEX AND LOVE,243413,spotify:artist:7qG3b048QCHVRO5Pv1T5lw,spotify:track:32lm3769IRfcnrQV11LO4E,spotify:album:2kZkiVn1m00XcgaWlLb2LD,0.04260,...,-3.503,1,0.1080,91.017,4,0.961,Lovey.,44,45,42
1428,153929,13,Fetty Wap,679 (feat. Remy Boyz),679 (feat. Remy Boyz),196693,spotify:artist:6PXS4YHDkKvl1wkIl4V8DL,spotify:track:5NQbUaeTEOGdD6hHcre0dZ,spotify:album:0TyDTzG2zt2tg0wONT9wSU,0.00256,...,-5.738,1,0.3180,190.050,4,0.603,All songs,66,83,41
2807,241546,66,Daughtry,It's Not Over,Daughtry (Deluxe Edition),215026,spotify:artist:5P5FTygHyx2G57oszR3Wot,spotify:track:234RqTZmnDTnxWlVciXHLD,spotify:album:6ii8Ja21xLGkFH8CQ6bKjj,0.05030,...,-3.245,0,0.0618,145.927,4,0.294,good old songs,99,122,82
1103,97870,3,Alexandra Stan,Mr. Saxobeat - Radio Edit,Saxobeats,195280,spotify:artist:0BmLNz4nSLfoWYW1cYsElL,spotify:track:4zIaXqKCXZs02eB6jEe5Mf,spotify:album:11UM41JGr8BavupgcGCICa,0.02390,...,-4.162,0,0.0527,127.047,4,0.745,Gym,6,6,6
1130,107065,10,Digital Farm Animals,Millionaire - Alan Walker Remix,Millionaire,189346,spotify:artist:5fyDppLDl1juIu1BcUT5zh,spotify:track:5HzbxweVAtisCrpoOF11qt,spotify:album:05XuMZb14sDGgreCf3Wu6F,0.01860,...,-4.577,1,0.0545,96.044,4,0.424,feels,11,11,11


(3079, 28)

In [84]:
def cos_similar_songs_playlist(cos_matrix, orig_df, target_playlist_inx, cand_list_size):
    """
    Input:
    cos_matrix: cosine matrix of the tracks
    orig_df: original df with tracks as rows, but with playlistid and other features (e.g., train)
    target_playlist_inx: index of playlist in the training set
    cand_list_size: candidate list of songs to recommend size (= test-set size * 15)
    
    Output:
    k_song_to_recommend: the most similar tracks per track
    """
    target_playlistid = train.loc[target_playlist_inx, "Playlistid"]
    target_track_inx = np.where(train["Playlistid"] == target_playlistid)
    
    # remove rows in matrix with songs that live in the same playlist with the target song
    # candidate_cos_matrix = np.delete(cos_matrix, target_track_inx, 1) # remoe songs that co-exist in the same playlist
    candidate_cos_matrix = cos_matrix
    
    ## For each song in the playlist, find k similar songs
    cand_list = []
    # cand_list_size = k*15
    # taget_track_inx
    k = np.floor(cand_list_size/len(target_track_inx)) # round(cand_list_size/len(target_track_inx))
    k_rest = cand_list_size - k*len(target_track_inx)
    # e.g., for a candidate list size of 30, get 3 songs for each track first
    for inx, i in enumerate(target_track_inx):
        candidate_song_rec = candidate_cos_matrix[target_inx, ]
        candidate_song_rec_inx = np.argsort(candidate_song_rec)
        unique_candidate_song_sorted = train['Track_uri'][candidate_song_rec_inx][::-1].drop_duplicates()
        tracks_in_target_playlist = train.loc[train["Playlistid"] == target_playlistid, "Track_uri"]
        song_to_recommend = np.array(unique_candidate_song_sorted.loc[~unique_candidate_song_sorted.isin(tracks_in_target_playlist)])
        
        if (k_rest !=0 & inx <= k_rest): # 30-24 = 6; for the first 6 tracks recommend k + 1 songs
            k_song_to_recommend = song_to_recommend[:int(k+1)]
        else:
            k_song_to_recommend = song_to_recommend[:int(k)]
        cand_list.append(k_song_to_recommend)

    return cand_list

In [76]:
len(np.where(train["Playlistid"] == 61388)), len(np.where(val["Playlistid"] == 61388)), len(np.where(test["Playlistid"] == 61388))

(1, 1, 1)

In [79]:
song_to_recommend[:15.0]

TypeError: slice indices must be integers or None or have an __index__ method

In [81]:
cand_size = len(np.where(val["Playlistid"] == target_playlistid)) * 15
trial = cos_similar_songs_playlist(train_scaled_cos_matrix, train, 0, cand_size)

<class 'numpy.ndarray'>
15.0


In [82]:
trial

[array(['spotify:track:7vRriwrloYVaoAe3a9wJHe',
        'spotify:track:22DKsoYFV5npPXmnPpXL7i',
        'spotify:track:00LfFm08VWeZwB0Zlm24AT',
        'spotify:track:6euh6chaRsi3Ywb1gA9LlO',
        'spotify:track:0QpYkajexWrB0P3TWvkHlm',
        'spotify:track:4FHu9bOzBjZurx89CMa42L',
        'spotify:track:1k1Bqnv2R0uJXQN4u6LKYt',
        'spotify:track:2Kbxfq8wkjMCno1QJe4yyw',
        'spotify:track:7cNz65PfCatRXoX7QtqM2A',
        'spotify:track:0osPUefhvYxoB2eZw6prBt',
        'spotify:track:3lKqtKFnN1Xi4W69YDN9PB',
        'spotify:track:3pndPhlQWjuSoXhcIIdBjv',
        'spotify:track:5dLz5CRzW8RcNEptn0NtOi',
        'spotify:track:5tz69p7tJuGPeMGwNTxYuV',
        'spotify:track:31Q9ZTF9x81BDonlObCbvP'], dtype=object)]

In [39]:
train.head()

,Playlistid,Trackid,Artist_Name,Track_Name,Album_Name,Track_Duration,Artist_uri,Track_uri,Album_uri,acousticness,...,loudness,mode,speechiness,tempo,time_signature,valence,Playlist,Album,Track,Artist
0,61388,35,Drake,Charged Up,Charged Up,189768,spotify:artist:3TVXtAsR1Inumwj472S9r4,spotify:track:4pc01CAhGKx15PY23uPjHp,spotify:album:2b3q6C2zmtXRKlk0j4eX4v,0.90900,...,-7.059,0,0.5530,84.762,4,0.308,OVO,39,95,18
1,51590,25,Roy Woods,Get You Good,Exis,226632,spotify:artist:7mDU6nMUJnOSY2Hkjz5oqM,spotify:track:2yTTfOaptQVJiocgjoQFfi,spotify:album:2vnPwMLtMXZWGkKsoylsGt,0.03060,...,-7.077,0,0.0638,86.185,4,0.329,Rap,30,38,23
2,193450,42,Major Lazer,Lean On (feat. MØ & DJ Snake),Peace Is The Mission,176561,spotify:artist:738wLrAtLtCtFOLvQBXOXp,spotify:track:4WjH9Bzt3kx7z8kl0awxh4,spotify:album:2XBnxKeRZi76u2iyGcMych,0.00346,...,-3.081,0,0.0625,98.007,4,0.274,Fall 2016,51,55,45
3,198885,12,Hozier,Take Me To Church,Hozier,241688,spotify:artist:2FXC3k01G6Gw61bmprjgqS,spotify:track:0IqKeD8ZSP72KbGYyzEcAs,spotify:album:36k5aXpxffjVGcNce12GLZ,0.63400,...,-5.303,0,0.0464,128.945,4,0.437,my favorite music,22,27,18
4,73524,0,Justin Timberlake,What Goes Around.../...Comes Around (Interlude),FutureSex/LoveSounds,448533,spotify:artist:31TPClRtHm23RisEBtV3X7,spotify:track:4VZmWrPYFJP8T17AciiQJh,spotify:album:3EaBZNWtqfr7Ju5iTrctnB,0.13100,...,-4.743,0,0.1350,76.007,4,0.401,Barry,6,8,4


In [40]:
target_track_inx = np.where(train["Playlistid"] == target_playlistid)

In [41]:
target_track_inx

(array([   0,   70,   95,  122,  127,  133,  135,  185,  290,  318,  380,
         440,  486,  488,  499,  524,  537,  542,  636,  652,  706,  733,
         739,  754,  791, 1065, 1073, 1087, 1097, 1139, 1156, 1227, 1241,
        1263, 1266, 1299, 1325, 1358, 1462, 1482, 1484, 1502, 1526, 1561,
        1575, 1579, 1616, 1622, 1623, 1664, 1672, 1673, 1745, 1754, 1763,
        1781, 1790, 1855, 1865, 1923, 1954]),)

In [30]:
cos_matrix = train_scaled_cos_matrix
orig_df = train
target_track_inx = 0 # target playlist inx
k = 5

trial_songs = cos_similar_songs_track(train_scaled_cos_matrix, train, 0, 5)

In [94]:
trial_songs

array(['spotify:track:7vRriwrloYVaoAe3a9wJHe',
       'spotify:track:22DKsoYFV5npPXmnPpXL7i',
       'spotify:track:00LfFm08VWeZwB0Zlm24AT',
       'spotify:track:6euh6chaRsi3Ywb1gA9LlO',
       'spotify:track:0QpYkajexWrB0P3TWvkHlm'], dtype=object)

In [33]:
unique_playlistid = train['Playlistid'].drop_duplicates()

In [34]:
unique_playlistid

0        61388
1        51590
2       193450
3       198885
4        73524
5       223534
6       231023
7         2259
8       175237
9       100221
10      256979
12      206080
13      208692
14      151474
15        2535
17       67941
18      114695
20      210308
21       68119
22      221044
23       38828
25       59745
27      117841
28      107065
29      190503
30      211406
31      190574
32      118724
33      229646
35       20043
         ...  
150     196206
154     242823
156     216556
161     123429
162     219212
163     159326
186     120613
187      81542
191     248553
213     252197
224      11119
226     107941
250      10254
252      97870
255      46047
257      77069
300     268318
308     208779
327      90187
335      48330
341     196255
345     182533
379     232421
381     238096
410      62657
434      41347
625     155081
664     161016
946     217189
1130     79469
Name: Playlistid, Length: 100, dtype: int64

In [ ]:
# Train kNN model
col_filter = NearestNeighbors(metric='cosine', algorithm='brute')
col_filter.fit(co_mat_sparse)

## Making Predictions

In [119]:
def nholdout(playlist_id, df):
    '''Pass in a playlist id to get number of songs held out in val/test set'''
    
    return len(df[df.Playlistid == playlist_id].Track_uri)

In [126]:
### Prediction Example
pi = 430 # target playlist index
kpreds = cos_similar_songs_playlist(train_scaled_cos_matrix, train, pi, nholdout(pi, val)*15)[0]

In [127]:
val[val.Playlistid == pi]

,Playlistid,Trackid,Artist_Name,Track_Name,Album_Name,Track_Duration,Artist_uri,Track_uri,Album_uri,acousticness,...,loudness,mode,speechiness,tempo,time_signature,valence,Playlist,Album,Track,Artist
20,430,20,Jesse L. Martin,I'll Cover You - Reprise,RENT (Original Motion Picture Soundtrack),219386,spotify:artist:5jpma87NOuRoh5FMf0CW5I,spotify:track:3ECCCziRCrTy4Sv3kOiS94,spotify:album:7JR7tGOAvqFSpVmDlCzHIJ,0.8310,...,-7.342,1,0.0406,79.902,4,0.181,Rent,2,28,11
12,430,12,Wilson Jermaine Heredia,I'll Cover You,RENT (Original Motion Picture Soundtrack),149773,spotify:artist:2wpskShaCrgkVcET4JzTAV,spotify:track:3HMhPsOVZXZZRbZKx4f2V0,spotify:album:7JR7tGOAvqFSpVmDlCzHIJ,0.2430,...,-3.347,1,0.0438,111.847,4,0.778,Rent,2,28,11
22,430,22,Rosario Dawson,Goodbye Love,RENT (Original Motion Picture Soundtrack),384853,spotify:artist:5gGhRZgP9BJL9fsFdvTzeX,spotify:track:5XfbSyRMdop6Te9znNL238,spotify:album:7JR7tGOAvqFSpVmDlCzHIJ,0.8100,...,-6.516,0,0.0491,87.230,4,0.151,Rent,2,28,11
9,430,9,Adam Pascal,Another Day,RENT (Original Motion Picture Soundtrack),286400,spotify:artist:0bvwPCit9Yh0yVdLKGnXRi,spotify:track:0CtHgpZKOCRdAHYYXsmEhJ,spotify:album:7JR7tGOAvqFSpVmDlCzHIJ,0.0421,...,-2.985,1,0.0427,143.646,4,0.455,Rent,2,28,11


In [128]:
val_set = val[val.Playlistid == pi]
val_set = val_set['Track_uri'] # ground truth

## Metrics

In [129]:
def r_precision(prediction, val_set):
# prediction should be a list of predictions
# val_set should be pandas Series of ground truths
    score = np.sum(val_set.isin(prediction))/val_set.shape[0]
    return score

In [130]:
kpreds

array(['spotify:track:4pc01CAhGKx15PY23uPjHp',
       'spotify:track:7vRriwrloYVaoAe3a9wJHe',
       'spotify:track:4jtyUzZm9WLc2AdaJ1dso7',
       'spotify:track:22DKsoYFV5npPXmnPpXL7i',
       'spotify:track:00LfFm08VWeZwB0Zlm24AT',
       'spotify:track:6euh6chaRsi3Ywb1gA9LlO',
       'spotify:track:0QpYkajexWrB0P3TWvkHlm',
       'spotify:track:2durxb17bXcmQJHSt8JAdO',
       'spotify:track:4FHu9bOzBjZurx89CMa42L',
       'spotify:track:1k1Bqnv2R0uJXQN4u6LKYt',
       'spotify:track:7cNz65PfCatRXoX7QtqM2A',
       'spotify:track:0osPUefhvYxoB2eZw6prBt',
       'spotify:track:3lKqtKFnN1Xi4W69YDN9PB',
       'spotify:track:3pndPhlQWjuSoXhcIIdBjv',
       'spotify:track:5dLz5CRzW8RcNEptn0NtOi',
       'spotify:track:03L2AoiRbWhvt7BDMx1jUB',
       'spotify:track:5tz69p7tJuGPeMGwNTxYuV',
       'spotify:track:31Q9ZTF9x81BDonlObCbvP',
       'spotify:track:3ZLyt2ndLFBh148XRYjYYZ',
       'spotify:track:2G9lekfCh83S0lt2yfffBz',
       'spotify:track:68EMU2RD1ECNeOeJ5qAXCV',
       'spoti

In [131]:
### Example Usage
r_precision(kpreds, val_set)

0.0

In [132]:
### NDCG Code Source: https://gist.github.com/bwhite/3726239
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [133]:
### Example Usage
# Generate binary relevance array
r = np.zeros(len(kpreds))
for i, p in enumerate(kpreds):
    if p in val_set:
        r[i] = 1

ndcg_at_k(r, len(r))

0.0

## Baseline Model Performance

In [112]:
unique_playlistid

0        61388
1        51590
2       193450
3       198885
4        73524
5       223534
6       231023
7         2259
8       175237
9       100221
10      256979
12      206080
13      208692
14      151474
15        2535
17       67941
18      114695
20      210308
21       68119
22      221044
23       38828
25       59745
27      117841
28      107065
29      190503
30      211406
31      190574
32      118724
33      229646
35       20043
         ...  
150     196206
154     242823
156     216556
161     123429
162     219212
163     159326
186     120613
187      81542
191     248553
213     252197
224      11119
226     107941
250      10254
252      97870
255      46047
257      77069
300     268318
308     208779
327      90187
335      48330
341     196255
345     182533
379     232421
381     238096
410      62657
434      41347
625     155081
664     161016
946     217189
1130     79469
Name: Playlistid, Length: 100, dtype: int64

In [141]:
rps = []
ndcgs = []
for pid in unique_playlistid: # loop through each playlist
    print(pid)
    ps = cos_similar_songs_playlist(train_scaled_cos_matrix, train, pi, nholdout(pi, val)*50)[0]# predictions
    vs = val[val.Playlistid == pid].Track_uri # ground truth
    rps.append(r_precision(ps, vs)) # append individual r-precision score
    
    # NDCG
    r = np.zeros(len(ps))
    for i, p in enumerate(ps):
        if np.any(vs.isin([p])):
            r[i] = 1
    ndcgs.append(ndcg_at_k(r, len(r)))
    

61388
51590
193450
198885
73524
223534
231023
2259
175237
100221
256979
206080
208692
151474
2535
67941
114695
210308
68119
221044
38828
59745
117841
107065
190503
211406
190574
118724
229646
20043
241349
622
430
92360
33568
4575
22204
56681
241546
168982
164819
155598
11136
181874
116737
118342
195247
120354
6238
151748
89825
174875
167436
249966
266156
186672
271274
201186
72703
89955
237553
1990
193016
58121
153929
230183
204668
37634
42049
194212
196206
242823
216556
123429
219212
159326
120613
81542
248553
252197
11119
107941
10254
97870
46047
77069
268318
208779
90187
48330
196255
182533
232421
238096
62657
41347
155081
161016
217189
79469


In [142]:
avg_rp = np.mean(rps)
avg_ndcg = np.mean(ndcgs)
print('Avg. R-Precision: ', avg_rp)
print('Avg. NDCG: ', avg_ndcg)
print('Total Sum: ', np.mean([avg_rp, avg_ndcg]))

Avg. R-Precision:  0.04360068526244997
Avg. NDCG:  0.031060419956590445
Total Sum:  0.03733055260952021
